In [1]:
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import numpy as np
import os
import cv2
import imutils
import time
import uuid
import random
from scipy import ndimage
from PIL import Image
from math import floor, ceil, pi
from glob import glob

In [7]:
def rotate_bound_2(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

In [2]:
def get_image_paths(folder):
    files = os.listdir(folder)
    files.sort()
    files = ['{}/{}'.format(folder, file) for file in files]
    return files

In [3]:
imgs_background = get_image_paths('/home/apofeo/Documents/testSet_full')
imgs_clp = get_image_paths('def_base/anpr_ocr/anpr_ocr__train/img')

In [4]:
print(len(imgs_background))

41000


In [5]:
IMG_SIZE = 200

In [17]:
s_img = cv2.imread(imgs_clp[1])
s_img = cv2.cvtColor(s_img, cv2.COLOR_RGB2RGBA)
# s_img = cv2.resize(s_img, (IMG_SIZE, IMG_SIZE))
s_img = ndimage.rotate(s_img, 135, reshape=True)
l_img = cv2.imread(imgs_background[29189])
l_img = cv2.resize(l_img, (IMG_SIZE, IMG_SIZE))
y_offset, x_offset = 0, 0
# x_offset=s_img.shape[0]//6
# y_offset=s_img.shape[1]//6
# s_img = cv2.imread("smaller_image.png", -1)

# cv2.imshow('l_img', l_img)
# cv2.waitKey(0)
# # [display]
# cv2.destroyAllWindows()

y1, y2 = y_offset, y_offset + s_img.shape[0]
x1, x2 = x_offset, x_offset + s_img.shape[1]

alpha_s = s_img[:, :, 3] / 255.0
alpha_l = 1.0 - alpha_s

for c in range(0, 3):
    l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] +
                              alpha_l * l_img[y1:y2, x1:x2, c])

crop_l_img = l_img[y1:y2, x1:x2]
cv2.imshow('l_img', crop_l_img)
cv2.waitKey(0)
# [display]
cv2.destroyAllWindows()

In [ ]:
# IMG_SIZE = 200
# for idx, each_img in enumerate(imgs_clp[:150]):
#     img_clp = cv2.imread(each_img, -1)
#     img_clp = cv2.cvtColor(img_clp, cv2.COLOR_RGB2RGBA)
    
#     for i in range(1, 361):
#         filename = uuid.uuid4().hex
#         background_index = random.randint(1,41001)
#         x_divider = random.randint(3,7)
#         y_divider = random.randint(3,7)
#         rotated = rotate_bound_2(img_clp, i)
#         img_background = cv2.imread(imgs_background[background_index], -1)
# #         img_background = cv2.cvtColor(img_background, cv2.COLOR_RGB2RGBA)
#         img_background = cv2.resize(img_background, (IMG_SIZE, IMG_SIZE))
        
#         x_offset=rotated.shape[0]//x_divider
#         y_offset=rotated.shape[1]//y_divider
        
#         img_background_edited = overlay_image_alpha(img_background,
#                         rotated[:, :, 0:3],
#                         (x_offset, y_offset),
#                         rotated[:, :, 3] / 255.0)
        
# #         img_background[y_offset:y_offset+rotated.shape[0], x_offset:x_offset+rotated.shape[1]] = rotated
        
#         if not os.path.exists(f'/home/apofeo/Documents/clp_rotated/train/{i}'):
#             os.makedirs(f'/home/apofeo/Documents/clp_rotated/train/{i}')
#         cv2.imwrite(f'/home/apofeo/Documents/clp_rotated/train/{i}/{filename}.png', img_background_edited)
        

In [7]:
import numpy as np
import cv2
def add_gaussian_noise(img, mean, var):
    img = img
    mean = mean
    var = var
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, (img.shape[0],img.shape[1]))

    noisy_image = np.zeros(img.shape, np.float32)

    if len(img.shape) == 2:
        noisy_image = img + gaussian
    else:
        noisy_image[:, :, 0] = img[:, :, 0] + gaussian
        noisy_image[:, :, 1] = img[:, :, 1] + gaussian
        noisy_image[:, :, 2] = img[:, :, 2] + gaussian

    cv2.normalize(noisy_image, noisy_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    noisy_image = noisy_image.astype(np.uint8)
    return noisy_image

# cv2.imshow('l_img', add_gaussian_noise(l_img,50,500))
# cv2.waitKey(0)
# # [display]
# cv2.destroyAllWindows()

In [36]:
# def overlay_image_alpha(img, img_overlay, pos, alpha_mask):
#     """Overlay img_overlay on top of img at the position specified by
#     pos and blend using alpha_mask.

#     Alpha mask must contain values within the range [0, 1] and be the
#     same size as img_overlay.
#     """

#     x, y = pos

#     # Image ranges
#     y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
#     x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

#     # Overlay ranges
#     y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
#     x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

#     # Exit if nothing to do
#     if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
#         return

#     channels = img.shape[2]

#     alpha = alpha_mask[y1o:y2o, x1o:x2o]
#     alpha_inv = 1.0 - alpha

#     for c in range(channels):
#         img[y1:y2, x1:x2, c] = (alpha * img_overlay[y1o:y2o, x1o:x2o, c] +
#                                 alpha_inv * img[y1:y2, x1:x2, c])
    
#     return img

# #     cv2.imshow('img', img)
# #     cv2.waitKey(0)
# #     # [display]
# #     cv2.destroyAllWindows()

In [28]:
# s_img = cv2.imread(imgs_clp[0], -1)
# s_img = cv2.cvtColor(s_img, cv2.COLOR_RGB2RGBA)
# s_img = rotate_bound_2(s_img, 45)
# l_img = cv2.imread(imgs_background[0], -1)
# l_img = cv2.resize(l_img, (IMG_SIZE, IMG_SIZE))

# x_offset=s_img.shape[0]//6
# y_offset=s_img.shape[1]//6

# img2 = overlay_image_alpha(l_img,
#                     s_img[:, :, 0:3],
#                     (x_offset, y_offset),
#                     s_img[:, :, 3] / 255.0)
# img2 = add_gaussian_noise(img2,50,3000)
# cv2.imshow('img2', img2)
# cv2.waitKey(0)
# # [display]
# cv2.destroyAllWindows()

#### Version 1

In [ ]:
start = time.time()

IMG_SIZE = 200
for idx, each_img in enumerate(imgs_clp[:150]):
    img_clp = cv2.imread(each_img)
    img_clp = cv2.cvtColor(img_clp, cv2.COLOR_RGB2RGBA)
    
    for i in range(1, 361):
        filename = uuid.uuid4().hex
        background_index = random.randrange(0,41000,1)
        x_divider = random.randint(3,7)
        y_divider = random.randint(3,7)
        noise_var = random.randrange(100,505,5)
        rotated = rotate_bound_2(img_clp, i)
        img_background = cv2.imread(imgs_background[background_index])
        img_background = cv2.resize(img_background, (IMG_SIZE, IMG_SIZE))
        
        x_offset=rotated.shape[0]//x_divider
        y_offset=rotated.shape[1]//y_divider
        
        y1, y2 = y_offset, y_offset + rotated.shape[0]
        x1, x2 = x_offset, x_offset + rotated.shape[1]

        alpha_s = rotated[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        try:
            for c in range(0, 3):
                img_background[y1:y2, x1:x2, c] = (alpha_s * rotated[:, :, c] +
                                      alpha_l * img_background[y1:y2, x1:x2, c])
        except Exception:
            print(f'background_index = {background_index}, angle = {i}, idx = {idx}')
        
#         img_background[y_offset:y_offset+rotated.shape[0], x_offset:x_offset+rotated.shape[1]] = rotated
        img_backgr_noise = add_gaussian_noise(img_background,50,noise_var)
    
        if not os.path.exists(f'/home/apofeo/Documents/clp_rotated/train/{i}'):
            os.makedirs(f'/home/apofeo/Documents/clp_rotated/train/{i}')
        cv2.imwrite(f'/home/apofeo/Documents/clp_rotated/train/{i}/{filename}.png', img_backgr_noise)

end = time.time()
print(end - start)

#### Don't delete under this cell

#### Version 2

In [8]:
start = time.time()

IMG_SIZE = 200
for idx, each_img in enumerate(imgs_clp[:100]):
    img_clp = cv2.imread(each_img)
    img_clp = cv2.cvtColor(img_clp, cv2.COLOR_RGB2RGBA)
    img_clp = cv2.resize(img_clp, (IMG_SIZE, IMG_SIZE))
    
    for i in range(1, 361):
        filename = uuid.uuid4().hex
        background_index = random.randrange(0,41000,1)
        noise_var = random.randrange(100,505,5)
        rotated = ndimage.rotate(img_clp, i, reshape=False)
        img_background = cv2.imread(imgs_background[background_index])
        img_background = cv2.resize(img_background, (IMG_SIZE, IMG_SIZE))
        
        x_offset, y_offset = 0, 0
        
        y1, y2 = y_offset, y_offset + rotated.shape[0]
        x1, x2 = x_offset, x_offset + rotated.shape[1]

        alpha_s = rotated[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        try:
            for c in range(0, 3):
                img_background[y1:y2, x1:x2, c] = (alpha_s * rotated[:, :, c] +
                                      alpha_l * img_background[y1:y2, x1:x2, c])
        except Exception:
            print(f'background_index = {background_index}, angle = {i}, idx = {idx}')
        
#         img_background[y_offset:y_offset+rotated.shape[0], x_offset:x_offset+rotated.shape[1]] = rotated
        img_backgr_noise = add_gaussian_noise(img_background,50,noise_var)
    
        if not os.path.exists(f'/home/apofeo/Documents/clp_rotated_v4/train/{i}'):
            os.makedirs(f'/home/apofeo/Documents/clp_rotated_v4/train/{i}')
        cv2.imwrite(f'/home/apofeo/Documents/clp_rotated_v4/train/{i}/{filename}.png', img_backgr_noise)

end = time.time()
print(end - start)

2804.472089290619


#### Version 3

In [9]:
start = time.time()

IMG_SIZE = 200
for idx, each_img in enumerate(imgs_clp[:100]):
    img_clp = cv2.imread(each_img)
    img_clp = cv2.cvtColor(img_clp, cv2.COLOR_RGB2RGBA)
    
    for i in range(1, 361):
        filename = uuid.uuid4().hex
        background_index = random.randrange(0,41000,1)
        noise_var = random.randrange(100,505,5)
        rotated = ndimage.rotate(img_clp, i, reshape=True)
        img_background = cv2.imread(imgs_background[background_index])
        img_background = cv2.resize(img_background, (IMG_SIZE, IMG_SIZE))
        
        x_offset, y_offset = 0, 0
        
        y1, y2 = y_offset, y_offset + rotated.shape[0]
        x1, x2 = x_offset, x_offset + rotated.shape[1]

        alpha_s = rotated[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        try:
            for c in range(0, 3):
                img_background[y1:y2, x1:x2, c] = (alpha_s * rotated[:, :, c] +
                                      alpha_l * img_background[y1:y2, x1:x2, c])
        except Exception:
            print(f'background_index = {background_index}, angle = {i}, idx = {idx}')
        
#         img_background[y_offset:y_offset+rotated.shape[0], x_offset:x_offset+rotated.shape[1]] = rotated
        img_backgr_noise = add_gaussian_noise(img_background,50,noise_var)
        crop_img_backgr_noise = img_backgr_noise[y1:y2, x1:x2]
    
        if not os.path.exists(f'/home/apofeo/Documents/clp_rotated_v5/train/{i}'):
            os.makedirs(f'/home/apofeo/Documents/clp_rotated_v5/train/{i}')
        cv2.imwrite(f'/home/apofeo/Documents/clp_rotated_v5/train/{i}/{filename}.png', crop_img_backgr_noise)

end = time.time()
print(end - start)

1879.8535912036896
